In [2]:
import sys
sys.path.append('/workspace/src/')


In [ ]:
#first get dois and corresponding full_text

from utils import gen_full_text_docling, gen_table
import ir_datasets
import pickle
from tqdm import tqdm

from dotenv import dotenv_values, load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from database.model import Base, Document, Table
from database.chunk_model import Chunk_Base, Chunk

from langchain_text_splitters import RecursiveCharacterTextSplitter
import langchain_core.documents

db_vals = dotenv_values("/workspace/src/.env")
load_dotenv()


In [3]:
engine = create_engine(f"postgresql+psycopg2://{db_vals['USER']}:{db_vals['PASSWORD']}@{db_vals['ADDRESS']}:{db_vals['PORT']}/{db_vals['DB']}", echo=False)
session = Session(engine)
Base.metadata.create_all(engine)

In [4]:
all_dois = list(set(doi[0] for doi in session.query(Document.doi).all()))

In [ ]:
BATCH_SIZE = 1000
doi_full_text = {}

for i in tqdm(range(0, len(all_dois), BATCH_SIZE)):
    batch = all_dois[i:i + BATCH_SIZE]
    docs = session.query(Document).filter(Document.doi.in_(batch)).all()
    for doc in docs:
        doi_full_text[doc.doi] = (doc.title, doc.full_text)

In [ ]:
full_texts = {}
for doi, doc in tqdm(doi_full_text.items()):
    full_texts[doi] = gen_full_text_docling(doi_full_text[doi])

In [ ]:
metadatas, texts = [doi for doi in sorted(full_texts.keys())], [full_texts[doi] for doi in sorted(full_texts.keys())]
print(len(metadatas), len(texts))

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,  # chunk size (characters)
    chunk_overlap=100,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

In [13]:
lang_docs = [langchain_core.documents.Document(page_content=text, metadata={"doi": doi}) for doi, text in full_texts.items()]

In [ ]:
all_splits = text_splitter.split_documents(lang_docs)

In [31]:
chunk_engine = create_engine(f"postgresql+psycopg2://{db_vals['USER']}:{db_vals['PASSWORD']}@{db_vals['ADDRESS']}:{db_vals['PORT']}/cord19chunks", echo=False)
chunk_session = Session(chunk_engine)
Chunk_Base.metadata.create_all(chunk_engine)

In [ ]:
#add splits to database
cnt = 0
for split in tqdm(all_splits):
    chunk = Chunk(doi=split.metadata["doi"], chunk_text=split.page_content, chunk_type="RCTS_512_100", modality_type="text")
    chunk_session.add(chunk)
    cnt += 1
    if cnt % 1000 == 0:
        chunk_session.commit()
chunk_session.commit()


In [ ]:
#chunk tables

In [5]:
all_table_dois = list(set(doi[0] for doi in session.query(Table.ir_tab_id).all()))

In [ ]:
BATCH_SIZE = 1000
doi_table = {}

for i in tqdm(range(0, len(all_table_dois), BATCH_SIZE)):
    batch = all_table_dois[i:i + BATCH_SIZE]
    docs = session.query(Table).filter(Table.ir_tab_id.in_(batch)).all()
    for doc in docs:
        doi_table[doc.ir_tab_id] = (doc.table_name, doc.header, doc.content, doc.caption, doc.references)

In [ ]:
full_tables = {}
for doi, doc in tqdm(doi_table.items()):
    full_tables[doi] = gen_table(doc)

In [35]:
table_splitter = RecursiveCharacterTextSplitter(
    chunk_size=8192,  
    chunk_overlap=1000, 
    add_start_index=True,  
)

In [ ]:
metadatas_t, texts_t = [doi for doi in sorted(full_tables.keys())], [full_tables[doi] for doi in sorted(full_tables.keys())]
print(len(metadatas_t), len(texts_t))

In [37]:
lang_tables= [langchain_core.documents.Document(page_content=text, metadata={"doi": doi}) for doi, text in full_tables.items()]

In [ ]:
all_table_splits = table_splitter.split_documents(lang_tables)

In [ ]:
cnt = 0
for split in tqdm(all_table_splits):
    chunk = Chunk(doi=split.metadata["doi"], chunk_text=split.page_content, chunk_type="RCTS_8192_1000", modality_type="table")
    chunk_session.add(chunk)
    cnt += 1
    if cnt % 1000 == 0: 
        chunk_session.commit()
chunk_session.commit()

In [34]:
#chunk_session.query(Chunk).filter(Chunk.modality_type == "table").delete(synchronize_session=False)
#chunk_session.commit()